In [1]:
import pandas as pd
from collections import Counter
df = pd.read_csv('scrapeddata.csv')


## Format Data

#### Take away list stuff and turn into an actual python list

In [2]:
rowcounter = 0
themelist = []
themeset = []
# turn strings that I scraped to actual lists 
for row in df.iterrows():
    #iterate over rows 
    selecteddata = df.iloc[rowcounter,:]
    
    #turn into string
    themes = selecteddata['Themes'].strip('[]')
    themes = themes.replace('"', '').split()
    themelist = themelist + themes
    
    #turn into an actual python list
    themeset = list(set(themeset + themelist))
    
    rowcounter += 1

#### Replace - with spaces

In [10]:
for items in themelist:
    for characters in items:
        characters.replace('-', ' ') #oof someone help pls

print(themelist)

['racism-and-slavery,', 'truth-and-justice,', 'family', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'family,', 'christianity', 'racism-and-slavery', 'racism-and-slavery,', 'truth-and-justice,', 'family', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'the-power-of-music', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'christianity,', 'the-power-of-music', 'racism-and-slavery,', 'truth-and-justice', 'racism-and-slavery,', 'truth-and-justice,', 'christianity', 'racism-and-slavery,', 'truth-and-justice,', 'christianity', 'racism-and-slavery,', 'truth-and-justice,', 'christianity', 'racism-and-slavery,', 'truth-and-justice', 'leadership,', 'idealism-vs-practicality,', 'colonial-society,', 'british-society', 'idealism-vs-practicality,', 'colonial-society', 'leadership,', 'british-society', 'militar

# Figure out most common themes

In [3]:
theme_srs = pd.Series(themelist).apply(
    lambda x: x.replace(",", "").replace("-", " ").strip()).value_counts()

In [4]:
def combine_themes(theme1, theme2, theme_srs):
    if theme1 in theme2 or theme2 in theme1:
        if theme_srs[theme1] > theme_srs[theme2]:
            return theme1
        else:
            return theme2
    else:
        return None

new_index = []
for i in range(len(theme_srs)):
    #print(i)
    theme1 = theme_srs.index[i]
    for j in range(i):
        theme2 = theme_srs.index[j]
        combined_theme = combine_themes(theme1, theme2, theme_srs)
        if combined_theme is not None:
            new_index.append(combined_theme)
            break
    if len(new_index) < i + 1:
        new_index.append(theme1)


In [5]:
theme_srs.index = new_index

In [6]:
combined_theme_srs = theme_srs.reset_index().groupby("index")[0].sum().sort_values(ascending=False)

In [7]:
# can change number of canonical themes
canonical_themes = combined_theme_srs.iloc[:45].index.drop("race and racism")

In [8]:
# top list of themes - canonical for the whole project
canonical_themes

Index(['family', 'identity', 'love', 'women', 'power', 'language', 'religion',
       'death', 'justice', 'class', 'storytelling', 'nature', 'racism',
       'education', 'survival', 'memory', 'freedom', 'coming of age',
       'morality', 'war', 'fate', 'gender roles', 'growing up', 'time',
       'isolation', 'guilt', 'christianity', 'friendship', 'marriage',
       'community', 'tradition', 'home', 'heroism', 'appearances',
       'fate and free will', 'america', 'work', 'redemption', 'perspective',
       'resistance', 'duality', 'corruption', 'secrecy', 'art'],
      dtype='object', name='index')

In [9]:
print(df.shape)
df.head()

(8202, 3)


,Boolean,Quotes,Themes
0,1,Thus far the history of my life presents nothi...,"[""racism-and-slavery"", ""truth-and-justice"", ""f..."
1,1,"The idea struck me as a prudent one, though I ...","[""racism-and-slavery"", ""truth-and-justice""]"
2,1,Though suspicions of Brown and Hamilton were n...,"[""racism-and-slavery"", ""truth-and-justice""]"
3,1,"So we passed, hand-cuffed and in silence, thro...","[""racism-and-slavery"", ""truth-and-justice""]"
4,1,My cup of sorrow was full to overflowing. Then...,"[""racism-and-slavery"", ""family"", ""christianity""]"


In [10]:
df.loc[0]

Boolean                                                    1
Quotes     Thus far the history of my life presents nothi...
Themes     ["racism-and-slavery", "truth-and-justice", "f...
Name: 0, dtype: object

In [11]:
df.loc[0]["Themes"]

'["racism-and-slavery", "truth-and-justice", "family"]'

In [12]:
df2 = pd.DataFrame()
for theme in canonical_themes:
    temp_df = pd.DataFrame()
    temp_df["Quotes"] = df["Quotes"]
    temp_df["theme"] = theme
    temp_df["theme_in_quote"] = df["Themes"].apply(lambda x: theme in x.replace("-", " "))
    df2 = df2.append(temp_df)

In [13]:
# almost ready to be fed into ML algorithms
df2

,Quotes,theme,theme_in_quote
0,Thus far the history of my life presents nothi...,family,True
1,"The idea struck me as a prudent one, though I ...",family,False
2,Though suspicions of Brown and Hamilton were n...,family,False
3,"So we passed, hand-cuffed and in silence, thro...",family,False
4,My cup of sorrow was full to overflowing. Then...,family,True
5,"He would make us hold up our heads, walk brisk...",family,False
6,The influences and associations that had alway...,family,True
7,"He was my master, entitled by law to my flesh ...",family,False
8,"I must toil day after day, endure abuse and ta...",family,False
9,Bent with excessive toil—actually suffering fo...,family,False


In [14]:
true_df = df2[df2["theme_in_quote"] == True]
false_df = df2[df2["theme_in_quote"] == False].sample(frac=0.05)
df3 = pd.DataFrame().append(true_df).append(false_df).reset_index(
    drop=True)

In [15]:
canonicallist = list(canonical_themes)

df3["theme_index"] = df3["theme"].apply(lambda x: canonicallist.index(x))

In [16]:
df3

,Quotes,theme,theme_in_quote,theme_index
0,Thus far the history of my life presents nothi...,family,True,0
1,My cup of sorrow was full to overflowing. Then...,family,True,0
2,The influences and associations that had alway...,family,True,0
3,"Well, a favorable day for a person is a fortun...",family,True,0
4,". . . it’s very unusual to me, marrying the br...",family,True,0
5,Marie-Laure twists the chimney of the miniatur...,family,True,0
6,He sweeps her hair back from her ears; he swin...,family,True,0
7,"“It’s not forever, Jutta. Two years, maybe. Ha...",family,True,0
8,“But I wasn’t trying to reach England. Or Pari...,family,True,0
9,"Mostly he misses Jutta: her loyalty, her obsti...",family,True,0


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer(
    stop_words="english", 
    ngram_range=(1, 1),
    max_df=1.0,
    min_df=0.001
    
)
X = vectorizer.fit_transform(df3["Quotes"])

In [19]:
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
X_df["theme_index"] = df3["theme_index"]

In [20]:
X_df.head()

,000,10,abandon,abandoned,ability,able,abroad,absence,absolute,absolutely,...,yield,yo,york,young,younger,youngest,youth,zero,zeus,theme_index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [21]:
X.shape

(29168, 3735)

In [22]:
Y_selected = df3["theme_in_quote"]
Y_selected

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
         ...  
29138    False
29139    False
29140    False
29141    False
29142    False
29143    False
29144    False
29145    False
29146    False
29147    False
29148    False
29149    False
29150    False
29151    False
29152    False
29153    False
29154    False
29155    False
29156    False
29157    False
29158    False
29159    False
29160    False
29161    False
29162    False
29163    False
29164    False
29165    False
29166    False
29167    False
Name: theme_in_quote, Length: 29168, dtype: bool

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_df, Y_selected, 
                                                    test_size=0.33, 
                                                    random_state=42)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

/home/nathanlee/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [26]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

In [27]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [28]:
y_predict = clf.predict(X_test)

In [35]:
score = accuracy_score(y_predict, y_test)
print(score)

0.5998337835030126


In [32]:
import pickle

pickle.dump(clf, open('model' + str(score), 'wb'))

In [1]:
"abc,".replace(",", "")

'abc'

In [6]:
refinedlist = []

for theme in themelist:
    refinedTheme = theme.replace(",", "").replace("-", " ")
    refinedlist.append(refinedTheme)

In [8]:
rankingdict = {}

for theme in refinedlist:
    if theme in rankingdict.keys():
        rankingdict[theme] += 1
    else:
        rankingdict[theme] = 1 # help

In [7]:
refinedlist

['racism and slavery',
 'truth and justice',
 'family',
 'racism and slavery',
 'truth and justice',
 'racism and slavery',
 'truth and justice',
 'racism and slavery',
 'truth and justice',
 'racism and slavery',
 'family',
 'christianity',
 'racism and slavery',
 'racism and slavery',
 'truth and justice',
 'family',
 'racism and slavery',
 'truth and justice',
 'racism and slavery',
 'truth and justice',
 'racism and slavery',
 'the power of music',
 'racism and slavery',
 'truth and justice',
 'racism and slavery',
 'christianity',
 'the power of music',
 'racism and slavery',
 'truth and justice',
 'racism and slavery',
 'truth and justice',
 'christianity',
 'racism and slavery',
 'truth and justice',
 'christianity',
 'racism and slavery',
 'truth and justice',
 'christianity',
 'racism and slavery',
 'truth and justice',
 'leadership',
 'idealism vs practicality',
 'colonial society',
 'british society',
 'idealism vs practicality',
 'colonial society',
 'leadership',
 'british

In [11]:
rankingdict

{'racism and slavery': 17,
 'truth and justice': 18,
 'family': 146,
 'christianity': 28,
 'the power of music': 2,
 'leadership': 17,
 'idealism vs practicality': 5,
 'colonial society': 2,
 'british society': 2,
 'military strategy': 1,
 'appearances and inner lives': 1,
 'identity belonging and coming of age': 12,
 'drawing writing and junior s cartoons': 3,
 'racism poverty and alcoholism': 9,
 'overlapping opposites': 6,
 'hope dreams and loss': 8,
 'confessions revenge and forgiveness': 2,
 'the middle class': 2,
 'materialism': 2,
 'gender roles': 70,
 'fortune': 1,
 'self actualization': 5,
 'romantic love': 21,
 'achievement and mortality': 6,
 'storytelling': 54,
 'plans change and growth': 9,
 'fate and justice': 1,
 'society and hypocrisy': 12,
 'freedom': 32,
 'religion and superstition': 1,
 'growing up': 65,
 'slavery and racism': 8,
 'war and its aftermath': 4,
 'fate and the gods': 5,
 'revenge': 18,
 'the rules of society': 15,
 'the failure of marriage': 9,
 'innocen

# Eliminate uncommon values

In [21]:
df

,Boolean,Quotes,Themes
0,1,Thus far the history of my life presents nothi...,"[""racism-and-slavery"", ""truth-and-justice"", ""f..."
1,1,"The idea struck me as a prudent one, though I ...","[""racism-and-slavery"", ""truth-and-justice""]"
2,1,Though suspicions of Brown and Hamilton were n...,"[""racism-and-slavery"", ""truth-and-justice""]"
3,1,"So we passed, hand-cuffed and in silence, thro...","[""racism-and-slavery"", ""truth-and-justice""]"
4,1,My cup of sorrow was full to overflowing. Then...,"[""racism-and-slavery"", ""family"", ""christianity""]"
5,1,"He would make us hold up our heads, walk brisk...","[""racism-and-slavery""]"
6,1,The influences and associations that had alway...,"[""racism-and-slavery"", ""truth-and-justice"", ""f..."
7,1,"He was my master, entitled by law to my flesh ...","[""racism-and-slavery"", ""truth-and-justice""]"
8,1,"I must toil day after day, endure abuse and ta...","[""racism-and-slavery"", ""truth-and-justice""]"
9,1,Bent with excessive toil—actually suffering fo...,"[""racism-and-slavery"", ""the-power-of-music""]"
